## Import the libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from os import listdir
from os.path import isfile, join
import glob
import re
import csv
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 75

## Load data

In [2]:
df = pd.read_csv('./dft-road-casualty-statistics-accident-1979-2020 (1).csv')
df.head()

C:\Users\lonew\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (0,2,15,16,35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,...,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
0,197901A11AD14,1979,01A11AD14,NaN,NaN,NaN,NaN,1,3,2,...,-1,1,8,1,-1,0,-1,-1,-1,-1
1,197901A1BAW34,1979,01A1BAW34,198460.00000,894000.00000,NaN,NaN,1,3,1,...,-1,4,8,3,-1,0,-1,-1,-1,-1
2,197901A1BFD77,1979,01A1BFD77,406380.00000,307000.00000,NaN,NaN,1,3,2,...,-1,4,8,3,-1,0,-1,-1,-1,-1
3,197901A1BGC20,1979,01A1BGC20,281680.00000,440000.00000,NaN,NaN,1,3,2,...,-1,4,8,3,-1,0,-1,-1,-1,-1
4,197901A1BGF95,1979,01A1BGF95,153960.00000,795000.00000,NaN,NaN,1,2,2,...,-1,4,3,3,-1,0,-1,-1,-1,-1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8602824 entries, 0 to 8602823
Data columns (total 36 columns):
 #   Column                                       Dtype  
---  ------                                       -----  
 0   accident_index                               object 
 1   accident_year                                int64  
 2   accident_reference                           object 
 3   location_easting_osgr                        float64
 4   location_northing_osgr                       float64
 5   longitude                                    float64
 6   latitude                                     float64
 7   police_force                                 int64  
 8   accident_severity                            int64  
 9   number_of_vehicles                           int64  
 10  number_of_casualties                         int64  
 11  date                                         object 
 12  day_of_week                                  int64  
 13  time        

In [4]:
# Missing values by column
missing = df.isnull().sum().sort_values(ascending=False)
missing = missing[missing!=0]
missing

longitude                 4887402
latitude                  4887402
location_easting_osgr       11621
location_northing_osgr      11621
speed_limit                    37
time                            8
date                            8
dtype: int64

In [5]:
accident_df = pd.DataFrame(df['accident_year'].value_counts()).sort_values(by=['accident_year'], ascending=False).reset_index().rename(columns={'index':'Year','accident_year':'Accidents'})

In [6]:
accident_df = accident_df.sort_values('Year')
    

In [104]:

fig = px.line(accident_df, x='Year', y='Accidents', markers=True, 
              title='<b> Annual Number of Accidents in the UK</b>', 
              labels={'Year':'<b>Year</b>', 'Accidents':'<b>Accidents</b>'})
fig.update_layout(yaxis_range=[50000,290000], plot_bgcolor="#edf0f5",width=1200,height=600)
fig.add_shape(
      type= 'line',
      x0= 2020, x1= 2020, y0= 0, y1= 300000,
      line=dict(
      color='lightseagreen',
      dash='dot',
      width=2,
      ) 
    )
fig.add_shape(
      type= 'line',
      x0= 2008, x1= 2008, y0= 0, y1= 300000,
      line=dict(
      color='darkorange',
      dash='dot',
      width=2,
      ) 
)
fig.add_shape(
      type= 'line',
      x0= 1981, x1= 1981, y0= 0, y1= 300000,
      line=dict(
      color='darkred',
      dash='dot',
      width=2,
      ) 
)
fig.add_annotation(x=2020.5, y=100000,
            text="Coronavirus Outbreak",
            showarrow=False,
            arrowhead=0, textangle=-90) 
    
fig.add_annotation(x=2008.5, y=100000,
            text="Financial Crisis",
            showarrow=False,
            arrowhead=0, textangle=-90) 

fig.add_annotation(x=1981.5, y=100000,
            text="Deep Recession",
            showarrow=False,
            arrowhead=0, textangle=-90) 
fig.show()

In [8]:
df[['day_of_week', 'time']].isna().sum()

day_of_week    0
time           8
dtype: int64

In [9]:
df['time'] = pd.to_datetime(df['time'])

In [10]:
df.groupby(df['day_of_week']).describe()['time']

KeyError: 'time'

In [ ]:
df.groupby(df['time'].dt.hour)['day_of_week'].describe()

,count,mean,std,min,25%,50%,75%,max
time,,,,,,,,
0.00000,151364.00000,4.04481,2.39906,1.00000,1.00000,4.00000,7.00000,7.00000
1.00000,101737.00000,4.05177,2.47149,1.00000,1.00000,4.00000,7.00000,7.00000
2.00000,81610.00000,4.06830,2.52624,1.00000,1.00000,4.00000,7.00000,7.00000
3.00000,50246.00000,3.95707,2.47282,1.00000,1.00000,4.00000,7.00000,7.00000
4.00000,36388.00000,3.96969,2.34022,1.00000,1.00000,4.00000,6.00000,7.00000
5.00000,56518.00000,4.05082,2.02077,1.00000,2.00000,4.00000,6.00000,7.00000
6.00000,119846.00000,4.07227,1.79357,1.00000,3.00000,4.00000,6.00000,7.00000
7.00000,334648.00000,4.06800,1.62778,1.00000,3.00000,4.00000,5.00000,7.00000
8.00000,591424.00000,4.07809,1.58131,1.00000,3.00000,4.00000,5.00000,7.00000
